In [ ]:
import numpy as np
from numpy import pi as π
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import firedrake
from firedrake import Constant, inner, grad, dx as dz, ds

Before we do anything with Firedrake, we're going to define what the surface temperature forcing should be.
Then we'll plot it in order to make sure it's doing what we want.
The signal we want is
$$T_{\text{surf}}(t) = \overline{T} + \delta T\cdot\sin\left(\frac{2\pi t}{1\text{ year}} + \phi\right)$$
where $\phi$ is a phase shift.
The mean temperature $\overline{T}$ will be -50C and the amplitude variation $\delta T$ will be +/-30C.
We want it to start at the minimum temperature, so the phase shift will be $3\pi/2$.

In [ ]:
year = 365.25 * 24 * 60 * 60
T_mean = -50.0
δT = 30.0
ϕ = 3 * π / 2

def surface_temperature(t):
    return T_mean + δT * np.sin(2 * π * t / year + ϕ)

In [ ]:
num_times = 256
ts = np.linspace(0.0, 2 * year, num_times + 1)
fig, ax = plt.subplots()
ax.set_xlabel("time (years)")
ax.set_ylabel("temperature (${}^\circ$C)")
ax.plot(ts / year, surface_temperature(ts));

Create an initial interval mesh with 16 cells.

In [ ]:
nz = 16
Lz = 200.0
mesh = firedrake.IntervalMesh(nz, Lz)

We'll use degree-1 continuous Galerkin (CG) finite elements to start.
Later we can try higher degree.

In [ ]:
element = firedrake.FiniteElement("CG", "interval", 1)
V = firedrake.FunctionSpace(mesh, element)

These variables are `Constant`, which means they don't depend on the spatial coordinate of the domain, but they can vary in time.
It's a good idea to wrap a lot of physical parameters in `Constant` because otherwise the raw floating point value gets hard-coded into the generated C code.

In [ ]:
T_basal = Constant(T_mean + δT)
T_surface = Constant(T_mean - δT)

The return value from the `SpatialCoordinate` function works similarly to a sympy symbol -- we can form more complex algebraic expressions out of it to use later.
Here we make the initial temperature a linear function of depth.

In [ ]:
z, = firedrake.SpatialCoordinate(mesh)

T = firedrake.Function(V)
lz = Constant(Lz)
T.interpolate(T_basal * (1 - z / lz) + T_surface * z / lz)

In [ ]:
fig, ax = plt.subplots()
firedrake.plot(T, axes=ax)
ax.set_xlabel("depth (m)")
ax.set_ylabel("temperature (${}^\circ$C)");

Next we'll choose the total time and timestep.
I picked these sort of arbitrarily.
We'll talk about how to choose them in class.

In [ ]:
final_time = 30 * year
timestep = year / 24
num_steps = int(final_time / timestep)

Some physical constants for ice.

In [ ]:
ρ = Constant(917.0)    # kg / m^3
c = Constant(2.18)     # kJ / kg C
k = Constant(2.22e-3)  # kW / m C

Now the weak form of the time-dependent problem: find (at each timestep) a temperature field $T$ such that, for all test functions $\psi$ in our basis set,
$$\int_0^{L_z}\left(\rho c(T - T_n)\cdot\psi + \delta t\cdot k\nabla T\cdot\nabla\psi\right)dz = 0$$
subject to the boundary conditions $T|_{z = 0} = T_{\text{basal}}$ and $T|_{z = L_z} = T_{\text{surface}}$.

In [ ]:
δt = Constant(timestep)

ψ = firedrake.TestFunction(V)
T_n = firedrake.Function(V)
T_n.assign(T)
F = (ρ * c * (T - T_n) * ψ + δt * k * inner(grad(T), grad(ψ))) * dz

Since we're fixing the values of $T$, the boundary conditions need to be imposed in the linear system (essential) as opposed to in the variational form (natural).
Here we create them explicitly.

In [ ]:
bc1 = firedrake.DirichletBC(V, T_basal, [1])
bc2 = firedrake.DirichletBC(V, T_surface, [2])
bcs = [bc1, bc2]

And the timestepping loop.
When we `.assign` a value to `T_surface`, the surface boundary condition object sees the change.

In [ ]:
Ts = [T.copy(deepcopy=True)]
t = 0.0
for step in range(num_steps):
    t += timestep
    T_surface.assign(surface_temperature(t))
    firedrake.solve(F == 0, T, bcs)
    T_n.assign(T)
    Ts.append(T.copy(deepcopy=True))

And a movie.
Do you think we had a good enough discretization to start?

In [ ]:
%%capture

fig, ax = plt.subplots()
firedrake.plot(Ts[0], axes=ax)

def animate(T):
    ax.cla()
    firedrake.plot(T, axes=ax)
    ax.set_ylim((T_mean - δT, T_mean + δT))
    ax.set_xlabel("depth (m)")
    ax.set_ylabel("temperature (${}^\circ$C)")

from matplotlib.animation import FuncAnimation
animation = FuncAnimation(fig, animate, Ts, interval=1e3/24)

In [ ]:
from IPython.display import HTML
HTML(animation.to_html5_video())

### Things to try

Wrap the code that creates the temperature time series in a Python function so that you can call it repeatedly with, say, different meshes or polynomial degrees.
Run this again with double the resolution (i.e. take `nz` to be 32 or 64 instead of 16).
Run it again with quadratic or cubic finite elements instead of linear elements.
Run it with a smaller timestep.
In whichever case you choose, compare the coarser to the finer solution somehow.

Try altering the boundary conditions to use, say, Robin conditions (proportional gradient) instead of Dirichlet (fixed value) conditions.
Generate a time series for both and see how they compare.
Use two different values of the exchange coefficient with Robin conditions and see how the solutions differ.